# Promises

A Promise is a placeholder for a value which isn't known yet.

Julia's Tasks can be viewed as Promises though because they are also generators you can't build abstractions on top of them that assume they are just simple values. RemoteRefs can also be viewed as a form of Promise but they don't have a nice API for the simple case since they are also intended to enable ongoing communication channels. Some people [really like](http://swannodette.github.io/2013/08/23/make-no-promises/) these sorts of bonus features. I don't.

In [1]:
@require "github.com/jkroso/Promises.jl" need @defer @thread;

If you have an expensive computation which isn't always needed a simple optimisation is to wrap it in a Deferred. This will prevent the computation from being executed until its actually needed.

In [2]:
@defer JSON.parse(readall("Readme.ipynb"))["metadata"]["language_info"]["name"]

/Users/jkroso/.julia/kip/github.com/jkroso/Promises.jl/tarball/fd1d516/main.jl.Deferred{Any}((anonymous function),:pending,#undef,#undef)

Then to unwrap it just call `need` on it

In [3]:
need(Out[2])

"julia"

In the above example we used a promise to represent the parsed version of a JSON file because we weren't sure if it would actually be used or not. But when it is needed it takes a long time to compute because of the IO operation to read from the file system. In this case we would be better to pass control to another thread while we wait. This is what `@thread` does for us.

In [4]:
@thread JSON.parse(readall("Readme.ipynb"))["metadata"]["language_info"]["name"]

/Users/jkroso/.julia/kip/github.com/jkroso/Promises.jl/tarball/fd1d516/main.jl.Result{Any}(Condition(Any[]),:evaled,"julia",#undef)

In [5]:
need(Out[4])

"julia"